In [ ]:
import numpy as np
import h5py
import os
from SpikeSorting import params 

In [ ]:
def create_templates(channels, waveforms, spike_clusters, unique_clusters):
    """
    Create average waveform templates for each cluster.
    
    Return: [n_clusters, n_samples, n_channels] 
    """
    n_clusters = len(unique_clusters)
    n_samples = waveforms.shape[1]
    # Estimate number of channels from waveform data
    n_channels = len(np.unique([np.argmax(np.abs(waveforms[i])) for i in range(len(waveforms))]))
    
    templates = np.zeros((n_clusters, n_samples, n_channels))
    
    for i, cluster_id in enumerate(unique_clusters):
        cluster_mask = (spike_clusters == cluster_id)
        cluster_waveforms = waveforms[cluster_mask]
        # Use the first channel where this cluster appears as main channel
        main_channel = channels[cluster_mask][0]
        
        if len(cluster_waveforms) > 0:
            mean_waveform = np.mean(cluster_waveforms, axis=0)
            templates[i, :, main_channel] = mean_waveform
    return templates

In [ ]:
def create_neuropixels_channel_positions(all_channels_positions, channel_map):
    """
    Extract positions for valid channels from full probe layout.
    
    Args:
        all_channels_positions: Complete probe layout [n_total_channels, 2]
        channel_map: Actual valid channel ID array
    """
    valid_positions = []
    for channel_id in channel_map:
        if 0 <= channel_id < len(all_channels_positions):
            valid_positions.append(all_channels_positions[channel_id])
        else:
            print(f"Warning: channel {channel_id} out of range, use default location")
            valid_positions.append([0, 0])
    
    return np.array(valid_positions, dtype=np.float32)

In [ ]:
def create_full_neuropixels_layout(n_channels):
    """
    Create standard Neuropixels 1.0 channel layout.
    
    MODIFY this function for different electrode geometries:
    - Adjust vertical_spacing, horizontal_spacing, row_offset for other probes
    - Change the layout pattern for non-Neuropixels probes
    
    Geometry:
    - Two staggered columns
    - Vertical spacing: 20µm
    - Horizontal spacing: 32µm 
    - Horizontal offset between adjacent rows: 16µm
    """
    positions = []
    
    # ADJUST these parameters for different probe types
    vertical_spacing = 20    # µm between rows
    horizontal_spacing = 32  # µm between columns  
    row_offset = 16          # µm horizontal shift for alternating rows
    
    for channel_id in range(n_channels):
        row = channel_id // 2
        col = channel_id % 2
        
        x = col * horizontal_spacing  
        y = row * vertical_spacing  
        
        # Stagger alternating rows
        if row % 2 == 1:
            x += row_offset
        
        positions.append([x, y])
    
    return np.array(positions, dtype=np.float32)

In [ ]:
def create_params_file(output_dir, n_channels, sample_rate):
    """
    Create params.py configuration file for Phy.
    
    MODIFY these parameters according to your data:
    - dat_path: Path to raw binary data file (if available)
    - dtype: Data type of raw recording ('int16', 'float32', etc.)
    - sample_rate: Actual sampling rate of your recording
    """
    params_content = f'''
    dat_path = '/spikesorting/neuropixel/continuous.dat'  # MODIFY: Path to raw binary data
    n_channels_dat = {n_channels}
    dtype = 'int16'  # MODIFY: Change if your raw data uses different data type
    sample_rate = {sample_rate}
    hp_filtered = False
    '''
    with open(os.path.join(output_dir, 'params.py'), 'w') as f:
        f.write(params_content)


In [ ]:
def create_cluster_groups(output_dir, unique_clusters):
    """
    Create initial cluster_group.tsv file.
    
    All clusters are initially marked as 'unsorted'.
    After manual curation in Phy, this will be updated to:
    'good', 'mua' (multi-unit), 'noise', etc.
    """
    import csv
    with open(os.path.join(output_dir, 'cluster_group.tsv'), 'w', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['cluster_id', 'group'])
        for cluster_id in unique_clusters:
            writer.writerow([cluster_id, 'unsorted'])

In [ ]:
def quick_convert(h5_file_path, output_dir='for_phy', n_channels=384, sample_rate=30000.0):
    """
    Convert custom H5 spike sorting results to Phy-compatible format.
    
    Args:
        h5_file_path: Path to input H5 file containing spike data
        output_dir: Output directory for Phy files
        n_channels: Total number of channels in the recording system
                   (MODIFY this if using different probe type)
        sample_rate: Sampling rate in Hz
    """
    
    os.makedirs(output_dir, exist_ok=True)
    group_name = 'spikeWindow_' + str(params.get('windowForTrain'))
    seed_name = f'seed_{params.get('seed')}'

    with h5py.File(h5_file_path, 'r') as f:
        windowInfo = f[group_name]
        spike_times = windowInfo['time'][:] 
        channel_map = windowInfo['electrode'][:]   
        waveforms = windowInfo['waveform'][:]         
                  
        sortInfo = windowInfo['sortInfo']
        seedInfo = sortInfo[seed_name]
        spike_clusters = seedInfo['pre_label'][:]      
        
        # Filter out noise clusters (cluster ID <= 0)
        indices_units = np.where(spike_clusters > 0)[0]
        units_c = channel_map[indices_units]
        units_l = spike_clusters[indices_units]
        units_t = spike_times[indices_units]
        units_w = waveforms[indices_units]

        # Save core spike data files
        np.save(f'{output_dir}/spike_times.npy', units_t.astype(np.int64))
        np.save(f'{output_dir}/spike_clusters.npy', units_l.astype(np.int32))

        # Create channel mapping - only include channels that actually have spikes
        unique_channels = np.unique(units_c)
        channel_map = unique_channels.astype(np.int32)
        np.save(os.path.join(output_dir, 'channel_map.npy'), channel_map)

        # ADJUST electrode geometry here for different probes
        full_layout = create_full_neuropixels_layout(n_channels)
        channel_positions = create_neuropixels_channel_positions(full_layout, channel_map)
        np.save(os.path.join(output_dir, 'channel_positions.npy'), channel_positions)

        # Create template waveforms for each cluster
        unique_clusters = np.unique(units_l)
        templates = create_templates(units_c, units_w, units_l, unique_clusters)
        np.save(os.path.join(output_dir, 'templates.npy'), templates.astype(np.float32))

        # Create configuration file
        create_params_file(output_dir, n_channels, sample_rate)

        # Create initial cluster groups (all unsorted)
        create_cluster_groups(output_dir, unique_clusters) 
    
    print(f"Convert successfully! The files are saved in Dir:{output_dir}.\n Use: cd {output_dir} && phy template-gui params.py")

In [ ]:
if __name__ == "__main__":
    quick_convert(
            "/spikesorting/neuropixel/results/spikeInfo.h5", # MODIFY for .h5 sorting results
            output_dir='for_phy', 
            n_channels=384,  # MODIFY for different probe types
            sample_rate=30000.0  # MODIFY to match your recording
        )